In [1]:
from pathlib import Path
from string import punctuation
import warnings

import numpy as np
# import psycopg2 as pg

import database
import extract_features

In [2]:
warnings.filterwarnings('ignore', category=RuntimeWarning)

In [3]:
from importlib import reload
reload(database)
reload(extract_features);

In [3]:
cwd = Path.cwd()
seg_npz = cwd / 'data' / 'seg_npz'
all_segments = sorted(seg for seg in seg_npz.iterdir() if seg.suffix == '.npz')

In [4]:
len(all_segments)

283

In [ ]:
from datetime import datetime
punct_table = str.maketrans({char: None for char in punctuation + ' '})

for i, segments_npz in enumerate(all_segments[225:], start=225):  # change limits!
    segments, name = np.load(segments_npz).values()
    name = np.asscalar(name).translate(punct_table)

    print(i, datetime.now().strftime("%A, %d. %B %Y %I:%M%p"), '\n')
    for seg_id, seg_audio in enumerate(segments):
        print(name, seg_id)
        feature_dct = extract_features.extract_all(seg_audio)
        database.insert_features(name, seg_id, feature_dct)
    print('='*20)

225 Monday, 11. February 2019 06:53PM 

ThisIsThePlace 0
ThisIsThePlace 1
ThisIsThePlace 2
ThisIsThePlace 3
ThisIsThePlace 4
ThisIsThePlace 5
ThisIsThePlace 6
ThisIsThePlace 7
226 Monday, 11. February 2019 07:00PM 

Dosed 0
Dosed 1
Dosed 2
Dosed 3
Dosed 4
Dosed 5
Dosed 6
Dosed 7
Dosed 8
Dosed 9
227 Monday, 11. February 2019 07:08PM 

DonTForgetMe 0
DonTForgetMe 1
DonTForgetMe 2
DonTForgetMe 3
DonTForgetMe 4
DonTForgetMe 5
DonTForgetMe 6
DonTForgetMe 7
DonTForgetMe 8
228 Monday, 11. February 2019 07:16PM 

TheZephyrSong 0
TheZephyrSong 1
TheZephyrSong 2
TheZephyrSong 3
TheZephyrSong 4
TheZephyrSong 5
TheZephyrSong 6
229 Monday, 11. February 2019 07:22PM 

CanTStop 0
CanTStop 1
CanTStop 2
CanTStop 3
CanTStop 4
CanTStop 5
CanTStop 6
CanTStop 7
230 Monday, 11. February 2019 07:29PM 

ICouldDieForYou 0
ICouldDieForYou 1
ICouldDieForYou 2
ICouldDieForYou 3
ICouldDieForYou 4
ICouldDieForYou 5
231 Monday, 11. February 2019 07:34PM 

Midnight 0
Midnight 1
Midnight 2
Midnight 3
Midnight 4
Midnig

158 NoChump 0-4

214 Easily 0-4